# Generate 3D mesh from point cloud

Based on the article [_"5-Step Guide to generate 3D meshes from point
clouds with Python"_
](https://towardsdatascience.com/5-step-guide-to-generate-3d-meshes-from-point-clouds-with-python-36bad397d8ba) by Florent Poux. In this notebook we will generate a 3D mesh for the Kermit dataset processed with [`linux-photogrammetry-tools`](https://github.com/epassaro/linux-photogrammetry-tools), so some steps could differ from the original source.

### Prerequisites

To run this notebook create a new `conda` environment:

```
$ conda create -n open3d -c open3d-admin -c conda-forge open3d=0.12
```

## 1. Load point cloud

In [ ]:
import os
import numpy as np
import open3d as o3d

In [ ]:
point_cloud = np.loadtxt(os.path.join("../examples/kermit/output/option-0000.ply"), skiprows=14)
output_path = os.path.join(".", "output")

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,3:6])
pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,6:-1]/255)

In [ ]:
#o3d.visualization.draw_geometries([pcd])

## 2.1 Meshing with ball pivoting algorithm (BPA)

We first compute the necessary radius parameter based on the average distances computed from all the distances between points:

In [ ]:
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3*avg_dist

In one command line, we can then create a mesh and store it in the `bpa_mesh` variable:

In [ ]:
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

Before exporting the mesh, we can downsample the result to an acceptable number of triangles, for example, 100k triangles:

In [ ]:
dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)

Additionally, if you think the mesh can present some weird artifacts, you can run the following commands to ensure its consistency:

In [ ]:
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

## 2.2 Meshing with Poisson surface reconstruction

To get results with Poisson, it is very straightforward. You just have to adjust the
parameters that you pass to the function as described above:

In [ ]:
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, \
                                                                         linear_fit=False)[0]

To get a clean result, it is often necessary to add a cropping step to clean unwanted artifacts. For this, we compute the initial bounding-box containing the raw point cloud, and we
use it to filter all surfaces from the mesh outside the bounding-box:

In [ ]:
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

## 3. Output and visualization

In [ ]:
os.makedirs(output_path, exist_ok=True)

In [ ]:
o3d.io.write_triangle_mesh(os.path.join(output_path, "bpa_mesh.ply"), dec_mesh)
o3d.io.write_triangle_mesh(os.path.join(output_path, "p_mesh.ply"), p_mesh_crop)

In [ ]:
def lod_mesh_export(mesh, lods, extension, path):
    
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(os.path.join(output_path, f"lod_{str(i)}{extension}"), mesh_lod)
        mesh_lods[i] = mesh_lod
    
    print("generation of "+str(i)+" LoD successful")
    
    return mesh_lods

In [ ]:
my_lods = lod_mesh_export(p_mesh_crop, [100000, 50000, 10000, 1000, 100], ".ply", output_path)
#my_lods2 = lod_mesh_export(bpa_mesh, [8000, 800, 300], ".ply", output_path)

In [ ]:
o3d.visualization.draw_geometries([my_lods[100000]])